In [6]:

import sys
import os
import pandas as pd
import torch
import torch.nn as nn
import numpy as np
import random
import matplotlib.pyplot as plt
from numpy import pi, matmul,sqrt,dot,array,zeros,cos,sin,pi,arccos
from torch.utils.data import DataLoader
from func import OR,heatplot,misorientation,ipfread,negsample
from Class import Data,Cluster,Dataset,CNN50a,CNN50b,CNN49a,CNN49b
import torch.nn.functional as F
import matplotlib.patches as patches
def properties(header=['ND面硬度(Hv)', 'TD面硬度(Hv)', 'UTS strain(%)', 'UTS stress(MPa)','Total Elongation(%)', 'E(GPa)', 'Ys(Mpa)']):
    df=pd.read_excel("data/properties.xlsx",sheet_name="summary")
    names=df["Title"].values
    data=df[header].values
    dic=dict()
    for i in range(data.shape[1]):
        data[:,i]=(data[:,i]-np.mean(data[:,i]))/np.std(data[:,i])
    for i,name in enumerate(names):
        dic[str(name)]=data[i,:].tolist()
    return dic


In [3]:
def Quaternion(angle):
    a,b,c=angle
    a=a/180*pi
    b=b/180*pi
    c=c/180*pi
    # a-> 1st(z); b->second(x'); c->third(z'')
    [cos(a/2)*cos(b/2)*cos(c/2)-sin(a/2)*cos(b/2)*sin(c/2),sin(a/2)*sin(b/2)*sin()
    return 

-1.0

In [7]:
feature=["Error","BS"]
pic_size=500
sam_size=49
prop_dic=properties()
def create_dataset(subset="train",steels=["780","980"],feature=["Orient","MAD"],prop_dic=prop_dic,pic_size=500,sam_size=50):
    bef=[]
    target=[]
    source=[]
    props=[]
    h=w=sam_size
    for steel in steels:
        corner=torch.load("data/%s_corner.pkl"%(steel))
        root="data/%s/%s/"%(steel,subset)
        for date in sorted(os.listdir(root)):
            path="%s%s/before/"%(root,date)
            print(path)
            data=Data(path)
            data.data["Orient"]=data.data["Orient"].reshape(data.h,data.w,-1)
            for (i,j) in corner[date]:
                app=[]
                for ele in feature:
                    if ele=="Orient":
                        app.append(data.data[ele][i:i+h,j:j+w])
                    else:   
                        app.append(data.data[ele][i:i+h,j:j+w,np.newaxis])
                bef.append(np.concatenate(app,axis=2))
                props.append(prop_dic[steel])
                source.append([path,(i,j)])
                target.append(0)
            for (i,j) in negsample(corner[date],pic_size,sam_size):
                app=[]
                for ele in feature:
                    if ele=="Orient":
                        app.append(data.data[ele][i:i+h,j:j+w])
                    else:
                        app.append(data.data[ele][i:i+h,j:j+w,np.newaxis])
                source.append([date,(i,j)])
                bef.append(np.concatenate(app,axis=2))
                props.append(prop_dic[steel])
                target.append(1)
    bef=np.transpose(np.array(bef),(0,3,1,2))
    props=np.array(props)
    target=np.array(target)
    return Dataset(bef,props,target,source)
train=create_dataset("train",feature=feature,pic_size=pic_size,sam_size=sam_size)
val=create_dataset("val",feature=feature,pic_size=pic_size,sam_size=sam_size)

data/780/train/20200916/before/
data/780/train/20200923/before/
data/780/train/20201019/before/
data/980/train/20210225/before/
data/980/train/20210304/before/
data/780/val/20200907/before/
data/980/val/20210302/before/


In [36]:
class CNN49ag(nn.Module):
    def __init__(self,in_channel,comp):
        super(CNN49ag, self).__init__()
        self.batchnorm = nn.BatchNorm2d(in_channel)
        self.relu = nn.ReLU() # activation
        self.maxpool = nn.MaxPool2d(kernel_size=2) 
        self.cnn1 = nn.Conv2d(in_channels=in_channel, out_channels=12, kernel_size=2, stride=1, padding=0) 
        self.cnn2 = nn.Conv2d(in_channels=12, out_channels=18, kernel_size=3, stride=1, padding=0) 
        self.cnn3 = nn.Conv2d(in_channels=18, out_channels=24, kernel_size=3, stride=1, padding=0) 
        self.fc1 = nn.Linear(24 * 9 * 9+comp, 2) 
        self.softmax = nn.Softmax(1)
    def forward(self, x,p):
        # Convolution 1 49
        out = self.batchnorm(x)
        out = self.cnn1(x) # 48
        out = self.relu(out)
        out = self.maxpool(out) #24
        out = self.cnn2(out) #22
        out = self.relu(out) 
        out = self.maxpool(out) #11
        out = self.cnn3(out) #9
        out = self.relu(out)
        out = out.view(out.size(0), -1)
        out = torch.cat([out,p],1)
        # Linear function (readout)
        out = self.fc1(out)
        out = self.softmax(out)
        return out

In [37]:
comp=len(train[0][1])
in_channel=train[0][0].shape[0]
model=CNN49ag(in_channel,comp)

In [38]:
train_loader= DataLoader(train, batch_size=64, shuffle=True,  num_workers=0, drop_last=True )
val_loader= DataLoader(val, batch_size=64, shuffle=True,  num_workers=0,  drop_last=False )


In [40]:
Loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
max_acc = 0.1
epoch = 200
for ep in range(epoch):
    for batch_ndx, sample in enumerate(train_loader):
        optimizer.zero_grad()
        X,P,Y,_= sample
        output=model(X,P)
        loss = Loss(output,Y)
        loss.backward()
        optimizer.step()
#         print("train ",ep,loss.item())
    model.eval()
    cum=0
    loss=0
    for batch_ndx, sample in enumerate(val_loader):
        X,P,Y,_= sample
        output=model(X,P)
        loss+=Loss(output,Y)
        predict = torch.max(output, 1)[1]
        cum+=np.sum((Y == predict).cpu().numpy())
    acc=cum/len(val)
    if acc>max_acc:
        max_acc=acc
        min_loss = loss.item()
        print(ep,min_loss)
        print("acc= ",cum,"/",len(val))
#         torch.save(model,"model/%.3f_%s_%s.pkl"%(acc,type(model).__name__,"_".join(feature)))

0 1.9317407608032227
acc=  103 / 164
2 1.9088889360427856
acc=  106 / 164
8 1.9466159343719482
acc=  108 / 164
43 1.9444442987442017
acc=  109 / 164
67 1.8517893552780151
acc=  110 / 164


In [44]:
len(train)

214

In [45]:
len(val)

136